In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 9.0 MB/s 
     |████████████████████████████████| 4.2 MB 55.8 MB/s 
     |████████████████████████████████| 1.2 MB 61.9 MB/s 
     |████████████████████████████████| 84 kB 4.3 MB/s 
     |████████████████████████████████| 596 kB 61.6 MB/s 
     |████████████████████████████████| 6.6 MB 56.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=8ec213ee0796de0fd6148fcf2b51c62c303380f70a920c5ded9fce4f6ea0be52
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# !nvidia-smi

Sun May 22 18:14:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# import torch
# torch.cuda.is_available()

True

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers import models, losses
from torch.utils.data import DataLoader

In [ ]:
# LM models (mBERT, XLM-R, etc.)
model_name = 'airesearch/wangchanberta-base-att-spm-uncased'
word_embedding_model = models.Transformer(model_name, max_seq_length=416)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),pooling_mode='cls')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/404M [00:00<?, ?B/s]

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertModel: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/883k [00:00<?, ?B/s]

In [ ]:
train_sentences = open('sentencepiece.txt').readlines()
# Convert train sentences to sentence pairs
train_data = [InputExample(texts=[s, s]) for s in train_sentences]
# DataLoader to batch your data
train_dataloader = DataLoader(train_data, batch_size=24, shuffle=True)

In [ ]:
# Use the denoising auto-encoder loss
train_loss = losses.MultipleNegativesRankingLoss(model)

In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=200,
    show_progress_bar=True,
    optimizer_params={'lr': 3e-5},
    output_path='output/simcse-model-wangchanberta',
    save_best_model=True,
    use_amp=True
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3541 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3541 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3541 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3541 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3541 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3541 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3541 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3541 [00:00<?, ?it/s]